This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and depends on Magma (tested using version
2.25-5).

In this notebook, we identify tetragonal curves of genus 7 with no $g^2_6$ which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 3 hours for completion.

In [1]:
load("preamble.sage")

Fix a smooth $(1,1)$-hypersurface $X_1$ in $\mathbf{P}^1 \times \mathbf{P}^3$ and find the $\mathbb{F}_{2^i}$-rational points on it for $i=1,2$.

In [2]:
F = GF(2)
P.<x0,x1,y0,y1,y2,y3> = PolynomialRing(F, 6)

In [3]:
gens1 = x0*y0 + x1*y1

In [4]:
S1 = [vector(t) for t in ProjectiveSpace(F, 1)]
S2 = [vector(t) for t in ProjectiveSpace(F, 3)]
for v in S1 + S2:
    v.set_immutable()
S0 = list(itertools.product(S1, S2))
S = [x for x in S0 if gens1(*x[0], *x[1]) == 0]
len(S)

21

Construct the subgroup of $\mathrm{GL}(2,\mathbb{F}_2) \times \mathrm{GL}(4, \mathbb{F}_2)$ fixing $X_1$, presented as a subgroup of $\mathrm{GL}(6, \mathbb{F}_2)$.

In [5]:
l0 = [block_matrix(2,2,[identity_matrix(4),0,0,g.matrix()], subdivide=False) for g in GL(2,F).gens()] +\
      [block_matrix(3,3,[Matrix([[0,1],[1,0]]),0,0,0,Matrix([[0,1],[1,0]]),0,0,0,identity_matrix(2)],subdivide=False)] +\
      [block_matrix(3,3,[Matrix([[1,1],[0,1]]),0,0,0,Matrix([[1,0],[1,1]]),0,0,0,identity_matrix(2)],subdivide=False)] +\
      [identity_matrix(6)]
l0[-1][5,3] = 1
G0 = GL(6,F).subgroup(l0)
G0.order()

576

Use an orbit lookup tree to find $\mathrm{GL}(2,\mathbb{F}_2) \times \mathrm{GL}(4, \mathbb{F}_2)$-orbit representatives for 7-tuples of $\mathbb{F}_2$-points in $\mathbf{P}^1 \times \mathbf{P}^3$. We forbid tuples in which some 5 points have the same image in $\mathbf{P}^1$.

In [6]:
def apply_group_elem(g, x):
    g1 = g.submatrix(nrows=2,ncols=2)
    g2 = g.submatrix(row=2,col=2)
    v1 = g1*x[0]
    v2 = g2*x[1]
    v1.set_immutable()
    v2.set_immutable()
    return (v1, v2)

In [7]:
assert all(apply_group_elem(g.matrix(),x) in S for g in G0.gens() for x in S)

In [8]:
def stabilizer(x):
    G1 = vec_stab(Matrix(F, x[0]), transpose=True)
    G2 = vec_stab(Matrix(F, x[1]), transpose=True)
    l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(4)], subdivide=False) for g in G1.gens()] + \
        [block_matrix(2,2,[identity_matrix(2),0,0,g.matrix()], subdivide=False) for g in G2.gens()]
    return GL(6, F).subgroup(l0)

In [9]:
def forbid(vecs, easy=False):
    if len(vecs) == 5:
        if len(set([x[0] for x in vecs])) == 1:
            return True    
    return False

In [10]:
def optimized_rep(g):
    return g.matrix()

In [11]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep,
           'forbid': forbid}
tree = build_orbit_tree(G0, S, 7, methods, verbose=False)

In [12]:
nodes = {6: green_nodes(tree, 6), 7: green_nodes(tree, 7)}
len(nodes[6]), len(nodes[7])

(254, 423)

For each orbit of 6-tuples or 7-tuples, find complete intersections of type $(1,1), (1,2), (1,2)$ passing through exactly these $\mathbb{F}_2$-rational points of $X_1$. We use the following facts.
- The second polynomial is only specified modulo the ideal generated by the first. Similarly, the third polynomial is only specified modulo the ideal generated by the first two.
- Given the first two hypersurfaces, every $\mathbb{F}_2$-point of $\mathbb{P}^1 \times \mathbb{P}^3$ lying on the intersection of the first two hypersurfaces but not belonging to our tuple defines an *affine* condition on the third hypersurface. We can thus find candidates for the third hypersurface by solving a system of linear equations.

In [13]:
monos11 = [prod(i) for i in itertools.product([x0,x1], [y0,y1,y2,y3])]
monos12 = [prod(i) for i in itertools.product([x0,x1], [prod(j) for j in itertools.combinations_with_replacement([y0,y1,y2,y3],2)])]

In [14]:
coords1 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos11)) for x in S}
coords2 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos12)) for x in S}

In [15]:
redundancy1 = [vector(F, ((gens1*y).coefficient(mu) for mu in monos12)) for y in [y0,y1,y2,y3]]
assert Matrix(redundancy1).rank() == 4
V1 = Matrix(F, [coords2[x] for x in S]).right_kernel()
assert all(w in V1 for w in redundancy1)

In [16]:
def redundancy(v1, redundancy1=redundancy1):
    return redundancy1 + [v1]

In [17]:
curves = defaultdict(list)
for s1 in [6, 7]:
    for vecs in nodes[s1]:
        d2 = {}
        V1a = Matrix([coords2[x] for x in vecs]).right_kernel()
        for (v1,) in subspaces_containing(V1a, V1a.subspace(redundancy1), 1, basis=True):
            gens2 = sum(v1[i]*monos12[i] for i in range(20))
            pts = [x for x in S if gens2(*x[0], *x[1]) == 0]
            pts2 = 0
            pts3 = 0
            perp = Matrix([coords2[x] for x in pts])
            target = vector(F, (0 if x in vecs else 1 for x in pts))
            for v2 in solve_right_iterator(perp, target, redundancy, v1):
                M = Matrix(F, [v1, v2]).echelon_form()
                if M in d2:
                    continue
                gens3 = sum(v2[i]*monos12[i] for i in range(20))
                d2[M] = (gens1, gens2, gens3)
        for M in d2:
            curves[(s1,)].append(d2[M])

In [18]:
[(s, len(curves[s])) for s in curves]

[((6,), 1565932), ((7,), 880486)]

Enforce the desired point counts over $\mathbb{F}_{2^i}$ for $i=2,3$ by point counting.

In [19]:
F4 = GF(4)
S14 = [vector(t) for t in ProjectiveSpace(F4, 1)]
S24 = [vector(t) for t in ProjectiveSpace(F4, 3)]
for v in S14 + S24:
    v.set_immutable()
S04 = list(itertools.product(S14, S24))
S4 = [x for x in S04 if gens1(*x[0], *x[1]) == 0]
len(S4)

105

In [20]:
tmp = [s for s in curves if len(s) == 1]
for (s1,) in tmp:
    gens2 = 0
    for gens in curves[(s1,)]:
        if gens2 != gens[1]:
            gens2 = gens[1]
            pts2 = [x for x in S4 if gens2(*x[0], *x[1]) == 0]
        gens3 = gens[2]
        s2 = sum(1 for x in pts2 if gens3(*x[0], *x[1]) == 0)
        if (s1 == 6 and s2 == 18) or (s1 == 7 and s2 == 15):
            curves[(s1,s2)].append((gens1, gens2, gens3))
    del curves[(s1,)]

In [21]:
[(s, len(curves[s])) for s in curves if len(s) == 2]

[((6, 18), 35454), ((7, 15), 157779)]

In [22]:
F8 = GF(8)
S18 = [vector(t) for t in ProjectiveSpace(F8, 1)]
S28 = [vector(t) for t in ProjectiveSpace(F8, 3)]
for v in S18 + S28:
    v.set_immutable()
S08 = list(itertools.product(S18, S28))
S8 = [x for x in S08 if gens1(*x[0], *x[1]) == 0]
len(S8)

657

In [23]:
tmp = [s for s in curves if len(s) == 2]
for (s1, s2) in tmp:
    gens2 = 0
    for gens in curves[(s1, s2)]:
        if gens2 != gens[1]:
            gens2 = gens[1]
            pts3 = [x for x in S8 if gens2(*x[0], *x[1]) == 0]
        gens3 = gens[2]
        s3 = sum(1 for x in pts3 if gens3(*x[0], *x[1]) == 0)
        if (s1 == 6 and s3 == 12) or (s1 == 7 and s3 == 7):
            curves[(s1,s2,s3)].append((gens1, gens2, gens3))
    del curves[(s1, s2)]

In [24]:
[(s, len(curves[s])) for s in curves if len(s) == 3]

[((6, 18, 12), 3749), ((7, 15, 7), 1884)]

Close out this case.

In [25]:
I1 = P.ideal([x0,x1])
I2 = P.ideal([y0,y1,y2,y3])
CR = magma.CoxRing(P, [I1, I2], [[1,1,0,0,0,0],[0,0,1,1,1,1]], [])
proj = CR.ToricVariety()

In [27]:
closeout(curves, X=proj, genus=7)

Number of curves found: 6
Number of isomorphism classes found: 1
No covers found in this case!
Total time: 190 minutes
